### Постановка задачи
Построить простую модель энергопотребления здания по среднему значению, оценить эффективность модели через метрику

\begin{equation}
RMSLE = {\sqrt{\sum_{i=1}^{n}{(log(p_i+1) - log(a_i+1))^2} \over n}}.
\end{equation}
* $n$ - число наблюдений
* $log$ - натуральный логарифм
* $p_i$ - вычисленное значение метрики
* $a_i$ - заданное значение метрики

Данные: http://video.ittensive.com/machine-learning/ashrae/train.0.0.csv.gz

Соревнование: https://www.kaggle.com/c/ashrae-energy-prediction/

© ITtensive, 2020

### Загрузка библиотек

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import rcParams
rcParams['figure.figsize'] = 12, 6

### Загрузка данных
Дополнительно сразу отсечем пустые дни и выделим час из значения времени

In [2]:
energy_0 = pd.read_csv('http://video.ittensive.com/machine-learning/ashrae/train.0.0.csv.gz', parse_dates=['timestamp'])
energy_0 = energy_0[energy_0['meter_reading'] > 0]
energy_0['hour'] = energy_0['timestamp'].dt.hour
energy_0.head()

,building_id,meter,timestamp,meter_reading,hour
704,0,0,2016-01-30 08:00:00,43.6839,8
725,0,0,2016-01-31 05:00:00,37.5408,5
737,0,0,2016-01-31 17:00:00,52.5571,17
2366,0,0,2016-04-08 14:00:00,59.3827,14
2923,0,0,2016-05-01 19:00:00,448.0000,19


### Разделение данных на обучение и проверку
Выделим 20% всех данных на проверку, остальные оставим на обучение

In [3]:
energy_0_train, energy_0_test = train_test_split(energy_0, test_size=0.2, random_state=11)
energy_0_train.head()

,building_id,meter,timestamp,meter_reading,hour
7536,0,0,2016-11-10 00:00:00,212.276,0
5979,0,0,2016-09-06 03:00:00,260.055,3
4834,0,0,2016-07-20 10:00:00,302.374,10
5952,0,0,2016-09-05 00:00:00,245.722,0
7272,0,0,2016-10-30 00:00:00,208.863,0


### Создадим модели
Среднее и медианное значение потребление энергии по часам

In [4]:
energy_0_train_hours = energy_0_train.groupby('hour')
energy_0_train_averages = pd.DataFrame({
    'Среднее': energy_0_train_hours['meter_reading'].mean(),
    'Медиана': energy_0_train_hours['meter_reading'].median()
})
energy_0_train_averages

,Среднее,Медиана
hour,,
0,238.346353,240.2610
1,239.851706,241.6260
2,238.782361,241.2850
3,240.474273,244.3570
4,237.618679,247.0870
5,239.917999,246.4040
6,236.623502,243.3325
7,240.269176,245.3805
8,242.533996,239.5790


### Функция проверки модели

\begin{equation}
RMSLE = {\sqrt{\sum_{i=1}^{n}{(log(p_i+1) - log(a_i+1))^2} \over n}}.
\end{equation}

Для вычисления метрики создадим шесть новых столбцов в тестовом наборе данных с:
1. логарифмом значения метрики,
2. предсказанием по среднему
3. предсказанием по медиане,
4. а также с квадратом разницы предсказаний
5. и логарифма значения.
6. Последний столбец добавим, чтобы сравнить предсказание с его отсутствием - нулями в значениях.

In [5]:
def calculate_model(x):
    meter_reading_log = np.log(x.meter_reading + 1)
    meter_reading_mean = np.log(energy_0_train_averages['Среднее'][x.hour] + 1)
    meter_reading_median = np.log(energy_0_train_averages['Медиана'][x.hour] + 1)
    x['meter_reading_mean_q'] = (meter_reading_log - meter_reading_mean)**2
    x['meter_reading_median_q'] = (meter_reading_log - meter_reading_median)**2
    x['meter_reading_zero_q'] = meter_reading_log**2
    return x


energy_0_test = energy_0_test.apply(calculate_model, axis='columns', result_type='expand')
energy_0_test.head()

,building_id,meter,timestamp,meter_reading,hour,meter_reading_mean_q,meter_reading_median_q,meter_reading_zero_q
5841,0,0,2016-08-31 09:00:00,241.626,9,0.000237,0.000658,30.156805
7796,0,0,2016-11-20 20:00:00,212.959,20,0.010068,0.011819,28.791642
4135,0,0,2016-06-21 07:00:00,219.784,7,0.007873,0.012032,29.129604
4799,0,0,2016-07-18 23:00:00,314.660,23,0.078547,0.074550,33.116177
4132,0,0,2016-06-21 04:00:00,204.085,4,0.022935,0.036235,28.338849


Теперь остается просуммировать квадраты расхождений, разделить на количество значений и извлечь квадратный корень

In [6]:
energy_0_test_median_rmsle = np.sqrt(energy_0_test['meter_reading_median_q'].sum() / len(energy_0_test))
energy_0_test_mean_rmsle = np.sqrt(energy_0_test['meter_reading_mean_q'].sum() / len(energy_0_test))
energy_0_test_zero_rmsle = np.sqrt(energy_0_test['meter_reading_zero_q'].sum() / len(energy_0_test))
print ("Качество медианы:", energy_0_test_median_rmsle)
print ("Качество среднего:", energy_0_test_mean_rmsle)
print ("Качество нуля:", energy_0_test_zero_rmsle)

Качество медианы: 0.26270467983785356
Качество среднего: 0.2605825449416217
Качество нуля: 5.449199522262051
